# Support Vector Machine Classifier

Equation of hyperplane: $y = wX + b$ <br>
Margin: $x_{1} - x_{2} = \frac{2}{||w||}$ <br>
Label, $ y_{i}=   \left\{
\begin{array}{ll}
      -1 & w^Tx_1-b \le -1 \\
      1 & w^Tx_1-b \ge 1 \\
\end{array} 
\right.  $

### Different Kernels for SVM
- Linear kernel: $K(x_1, x_2) = x_1^Tx_2$
- Polynomial kernel: $K(x_1, x_2) = (x_1^Tx_2 + r)^d$, where $d$ is the degree of the polynomial
- Radial-basis function: $K(x_1, x_2) = e^{-\gamma||x_1-x_2||^2}$
- Sigmoid kernel: $K(x_1, x_2) = tanh(\gamma x_1^Tx_2+r)$

### Loss function:
(Applicable for soft-margins)<br>
**Hinge-loss function**: $L(w, b) = max(0,1 - y_i(w^Tx_i - b))$

where 
- for misclassification, $L(w, b) = 1 - y_i(w^Tx_i - b)$
- for correct classification, $L(w, b) = 0 - y_i(w^Tx_i - b)$

### Gradient Descent
$w_{i+1} = w_i - \alpha \frac{\partial J}{\partial w}$ <br>
$b_{i+1} = b_i - \alpha \frac{\partial J}{\partial b}$ <br>

$ \frac{\partial J}{\partial w}=   \left\{
\begin{array}{ll}
      2\lambda w - y_i x_i & y_i(w^Tx_i-b) < 1 \\
      2\lambda w & y_i(w^Tx_i-b) \ge 1 \\
\end{array} 
\right.  $ <br>

$ \frac{\partial J}{\partial b}=   \left\{
\begin{array}{ll}
     y_i & y_i(w^Tx_i-b) < 1 \\
     0 & y_i(w^Tx_i-b) \ge 1 \\
\end{array} 
\right.  $ <br>

In [1]:
import numpy as np

In [2]:
class SVM_Classifier():
    def __init__(self, learning_rate, epochs, lambda_param):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.lambda_param = lambda_param

    def fit(self, X, y):
        self.m, self.n = X.shape
        
        self.w = np.zeros(self.n)
        self.b = 0

        self.X = X
        self.y = y

        for _ in range(self.epochs):
            self.update_weights()

    def update_weights(self):
        y_label = np.where(self.y <= 0, -1, 1)
        for index, x_i in enumerate(self.X):
            condition = y_label[index] * (np.dot(x_i, self.w) - self.b) >= 1
            if condition == True:
                dw = 2 * self.lambda_param * self.w
                db = 0
            else:
                dw = 2 * self.lambda_param * self.w - np.dot(x_i, y_label[index])
                db = y_label[index]
            
            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db

    def predict(self, X):
        output = np.dot(X, self.w) - self.b
        predicted_labels = np.sign(output)
        y_hat = np.where(predicted_labels <= -1, 0, 1)
        return y_hat

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
diabetes_data = pd.read_csv('Dataset/diabetes.csv')
diabetes_data.shape

(768, 9)

In [5]:
diabetes_data.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [6]:
features = diabetes_data.drop(columns=['Outcome'], axis=1)
target = diabetes_data['Outcome']

In [7]:
scaler = StandardScaler()
scaler.fit(features)
features = scaler.transform(features)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(614, 8) (154, 8) (614,) (154,)


In [9]:
model = SVM_Classifier(learning_rate=0.001, epochs=1000, lambda_param=0.01)

In [10]:
model.fit(X_train, y_train)

In [12]:
training_data_preds = model.predict(X_train)
training_accuracy = accuracy_score(y_train, training_data_preds)
print(f'Training accuracy: {training_accuracy}')

Training accuracy: 0.7768729641693811


In [13]:
testing_data_preds = model.predict(X_test)
testing_accuracy = accuracy_score(y_test, testing_data_preds)
print(f'Testing accuracy: {testing_accuracy}')

Testing accuracy: 0.7662337662337663
